<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [10]</a>'.</span>

In [1]:
# declare a list tasks whose products you want to use as inputs
upstream = None

In [2]:
# Parameters
query_params = {
    "rolling_window": 90,
    "search_terms": ["Tesla", "Tesla Inc", "Tesla Motors"],
    "gdelt_gkg_base_url": "http://data.gdeltproject.org/gdeltv2/",
}
product = {
    "nb": "/Users/aiujdm2/market_watch/output/notebooks/fetch_n_filter_gdelt_data.ipynb",
    "data": "/Users/aiujdm2/market_watch/output/data/raw/gdelt-gkg-tesla.csv",
}


In [3]:
import datetime
from pathlib import Path
from urllib.error import HTTPError
import gdelt
import pandas as pd
import os

import sys

import warnings

warnings.filterwarnings('ignore')

In [4]:
print(sys.executable)

/Users/aiujdm2/.local/share/virtualenvs/market_watch-dtlP-L11/bin/python


In [5]:
col_names = ['GKGRECORDID', 'DATE', 'SourceCollectionIdentifier', 'SourceCommonName',
             'DocumentIdentifier', 'Counts', 'V2Counts', 'Themes', 'V2Themes',
             'Locations', 'V2Locations', 'Persons', 'V2Persons', 'Organizations',
             'V2Organizations', 'V2Tone', 'Dates', 'GCAM', 'SharingImage',
             'RelatedImages', 'SocialImageEmbeds', 'SocialVideoEmbeds', 'Quotations',
             'AllNames', 'Amounts', 'TranslationInfo', 'Extras']

filter_cols = [
    'GKGRECORDID',
    'DATE',
    'SourceCollectionIdentifier',
    'SourceCommonName',
    'DocumentIdentifier',
    'V2Counts',
    'V2Themes',
    'V2Locations',
    'V2Persons',
    'V2Organizations',
    'V2Tone',
    'GCAM',
    'SharingImage',
    'RelatedImages',
    'SocialImageEmbeds',
    'SocialVideoEmbeds',
    'Quotations',
    'AllNames',
    'Amounts',
    'TranslationInfo',
    'Extras'
]
#  config parameter that tells us rolling window for getting data 
# i.e. a value of 30 in env.yaml means 30 days rolling window. We will fetch data from today to 30 days back.
rolling_window = query_params["rolling_window"]
gdelt_gkg_base_url = query_params["gdelt_gkg_base_url"]

In [6]:
gd = gdelt.gdelt(version=2)

In [7]:
from datetime import datetime, timedelta


def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta

def get_gkg_file_url(rolling_window, base_url):
    today = datetime.utcnow()

    pd_ts = pd.Timestamp(today)
    today_ts = pd_ts.round('15T')

    how_far_back = timedelta(days=rolling_window)
    start_date = today_ts - how_far_back
    for dt in datetime_range(start_date, today_ts,timedelta(minutes=15)):
            yield base_url + f'{dt.strftime("%Y%m%d%H%M00")}.gkg.csv.zip'
    
def get_urls(rolling_window, base_url):
    today = datetime.utcnow()

    pd_ts = pd.Timestamp(today)
    today_ts = pd_ts.round('15T')

    how_far_back = timedelta(days=rolling_window)
    start_date = today_ts - how_far_back
    urls = [
        base_url + f'{dt.strftime("%Y%m%d%H%M00")}.gkg.csv.zip'
        for dt in
        datetime_range(
            start_date,
            today_ts,
            timedelta(minutes=15)
        )
    ]
    return urls

In [8]:
urls = get_urls(
    rolling_window=rolling_window,
    base_url='http://data.gdeltproject.org/gdeltv2/'
)

In [9]:
def read_and_filter_files():
    
    data = []

    for url in get_gkg_file_url(rolling_window, base_url=gdelt_gkg_base_url):
        file_name = os.path.basename(url).split(".")[0]
        try:
            search_results = pd.read_csv(url, sep='\t', names=col_names, on_bad_lines='skip', encoding_errors='ignore')
        except HTTPError as err:
            print(f"Unable to fetch {url}. Encountered {err.code}")
            pass
        # Add search date to file name     
        file_path = product["data"]
        Path(file_path).parent.mkdir(exist_ok=True, parents=True)
        # Form a query friendly search terms     
        search_terms = "|".join(query_params['search_terms'])
        organizations = search_results['V2Organizations'].str.lower()
        filter_cond = organizations.str.contains(search_terms, regex=True, case=False, na=False)
        df = search_results[filter_cond]

        if len(df) > 0:
            print(f"==> Found {len(df)} matchign records out of {len(search_results)} for {search_terms} on {file_name}")
            data.append(df[filter_cols])
            
    data_df = pd.concat(data)
    data_df.to_csv(file_path) 
    print(f"Processed merged file with {len(data_df)} records")


    # # fetch data for each day and then filter it on search terms
# for url in urls:
#     file_name = os.path.basename(url).split(".")[0]
#     search_results = pd.read_csv(url, sep='\t', names=col_names, on_bad_lines='skip', encoding_errors='ignore')
#     # Add search date to file name     
#     file_path = f'{product["data"]}/{str(file_name)}'
#     Path(file_path).parent.mkdir(exist_ok=True, parents=True)

#     # Form a query friendly search terms     
#     search_terms = "|".join(query_params['search_terms'])
#     organizations = search_results['V2Organizations'].str.lower()
#     filter_cond = organizations.str.contains(search_terms, regex=True, case=False, na=False)
#     df = search_results[filter_cond]

#     if len(df) > 0:
#         print(f"==> Found {len(df)} matchign records out of {len(search_results)} for {search_terms} on {file_name}")
#         df[filter_cols].to_csv(file_path + '.csv')

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [10]:
read_and_filter_files()

==> Found 1 matchign records out of 1564 for Tesla|Tesla Inc|Tesla Motors on 20211221150000


==> Found 3 matchign records out of 1562 for Tesla|Tesla Inc|Tesla Motors on 20211221151500


==> Found 2 matchign records out of 1539 for Tesla|Tesla Inc|Tesla Motors on 20211221154500


==> Found 2 matchign records out of 1859 for Tesla|Tesla Inc|Tesla Motors on 20211221160000


==> Found 3 matchign records out of 1915 for Tesla|Tesla Inc|Tesla Motors on 20211221163000


...

==> Found 1 matchign records out of 1693 for Tesla|Tesla Inc|Tesla Motors on 20220117154500


==> Found 2 matchign records out of 1374 for Tesla|Tesla Inc|Tesla Motors on 20220117160000


==> Found 1 matchign records out of 1373 for Tesla|Tesla Inc|Tesla Motors on 20220117161500


==> Found 1 matchign records out of 1672 for Tesla|Tesla Inc|Tesla Motors on 20220117173000


URLError: <urlopen error [Errno 60] Operation timed out>

In [ ]:
# #  config parameter that tells us rolling window for getting data 
# # i.e. a value of 30 in env.yaml means 30 days rolling window. We will fetch data from today to 30 days back.
# rolling_window = query_params["rolling_window"]

# # fetch data for each day and then filter it on search terms
# for day_num in range(rolling_window):

#     today = datetime.today()
#     how_far_back = timedelta(days=day_num)
#     search_date = today - how_far_back

#     print(f"Searching for {search_date}")
#     # try:
#     search_results = gd.Search(
#         str(search_date),
#         table='gkg', 
#         coverage=False,
#         translation=True
#     )
#     # except:
#     #     print(f"=> No records returnd for date {search_date}")
#     #     continue;
#     print(search_results.columns)
#     break
#     print(f"=> Returned {len(search_results)} records")

#     # Add search date to file name     
#     file_path = f'{product["data"]}/{str(search_date)}'
#     Path(file_path).parent.mkdir(exist_ok=True, parents=True)

#     # Form a query friendly search terms     
#     search_terms = "|".join(query_params['search_terms'])
#     organizations = search_results['V2Organizations'].str.lower()
#     filter_cond = organizations.str.contains(search_terms, regex=True, case=False, na=False)
#     df = search_results[filter_cond]

#     if len(df) > 0 :
#         print(f"==> Found {len(df)} matchign records for {search_terms} on {search_date}")
#         df[filter_cols].to_csv(file_path + '.csv')